In [1]:
from telnetlib import X3PAD
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 

from sklearn.utils import shuffle
import pickle 
import sqlite3


In [2]:
df_2020 = pd.read_csv('merged_gw2020.csv',low_memory = False)
df_2021 = pd.read_csv('merged_gw2021.csv',low_memory = False)


In [3]:
season_2020 = pd.DataFrame({'season' : ['2019/20']*df_2020.shape[0]   })
season_2021 = pd.DataFrame({'season' : ['2020/21']*df_2021.shape[0]   })

In [4]:
df_2020 =pd.concat([df_2020, season_2020] ,axis = 'columns')
df_2021 =pd.concat([df_2021, season_2021] ,axis = 'columns')


In [5]:
df= pd.concat([df_2020 , df_2021], axis = 'index',ignore_index = True)


In [6]:
df["was_home"] = df["was_home"].astype(int)

In [7]:
df.sort_values(by=['name','season','GW'], inplace =True)
df.reset_index(inplace = True)


df.drop(['index'],axis= 'columns' , inplace =True)

In [8]:
df['id'] = df.groupby(['name']).ngroup()

In [9]:
df['pos']= df.groupby(['position']).ngroup()

In [10]:
df['teams'] = df.groupby(['team']).ngroup()

In [11]:
identity = df['id'].unique()

### Enter the gameweek 


In [12]:
x = 25

In [13]:

df_id = df.loc[: , ['id', 'name' ,'position']]
df_id.reset_index(inplace = True)
df_id.drop_duplicates(subset='id',keep = 'first',inplace = True)

df_id.drop(['index'],axis= 'columns' , inplace =True)
df_id

id                   name position
0        0         Aaron Connolly      FWD
62       1        Aaron Cresswell      DEF
125      2           Aaron Lennon      MID
144      3             Aaron Mooy      MID
182      4         Aaron Ramsdale       GK
...    ...                    ...      ...
39046  960              Zak Brunt      MID
39063  961  Zeze Steven Sessegnon      DEF
39101  962       Álvaro Fernández       GK
39125  963         Çaglar Söyüncü      DEF
39185  964           Ørjan Nyland       GK

[965 rows x 3 columns]

In [14]:
y=x
Gameweeks=[]

In [15]:
for i in range(5):
    Gameweeks.append(y)
    y=y-1


In [16]:
dataframes = {}

In [17]:
for GW in Gameweeks:
    filt=(df['GW'] == GW) & (df['season'] == '2020/21')
    dataframes[GW] = df[filt]
    dataframes[GW].reset_index(inplace = True)
    dataframes[GW].drop(['index'],axis= 'columns' , inplace =True)
    dataframes[GW].drop([ 'kickoff_time','season', 'name' , 'position','pos' ,'GW' ,'team','teams'],axis= 'columns' , inplace =True)
    

/home/aavash/Downloads/jupyter/environment/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
n = 0
for keys in dataframes:
    column = []
    
    for value in dataframes[keys].columns:
        
        if (value != 'id')  & (value!='team'):
            value = value +f'_{n}' 
        column.append(value)
        
    dataframes[keys].columns = column
    n=n-1   

In [19]:
for keys in dataframes:
    dataframes[keys].drop_duplicates(subset='id' , keep ='first' , inplace = True)
    dataframes[keys].reset_index(inplace = True)
    dataframes[keys].drop(['index'],axis= 'columns' , inplace =True)

/home/aavash/Downloads/jupyter/environment/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [20]:
dataframes[24].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   xP_-1                 634 non-null    float64
 1   assists_-1            634 non-null    int64  
 2   bonus_-1              634 non-null    int64  
 3   bps_-1                634 non-null    int64  
 4   clean_sheets_-1       634 non-null    int64  
 5   creativity_-1         634 non-null    float64
 6   element_-1            634 non-null    int64  
 7   fixture_-1            634 non-null    int64  
 8   goals_conceded_-1     634 non-null    int64  
 9   goals_scored_-1       634 non-null    int64  
 10  ict_index_-1          634 non-null    float64
 11  influence_-1          634 non-null    float64
 12  minutes_-1            634 non-null    int64  
 13  opponent_team_-1      634 non-null    int64  
 14  own_goals_-1          634 non-null    int64  
 15  penalties_missed_-1   6

In [21]:
df1 = pd.merge(dataframes[x] , dataframes[x-1], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-2], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-3], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-4], on = 'id' , how = 'outer')

In [22]:
filt = (df['GW'] == x) | (df['GW'] == x-1) | (df['GW'] == x-2) | (df['GW'] == x-3) | (df['GW'] == x-4)
df_position = df.loc[filt , ['id' , 'pos','teams']]

In [23]:
df_position.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_position.reset_index(inplace = True)
df_position.drop(['index'],axis= 'columns' , inplace =True)

In [24]:
df1.drop_duplicates(subset='id',keep = 'first',inplace = True)
df1.reset_index(inplace = True)
df1.drop(['index'],axis= 'columns' , inplace =True)

In [25]:
df1 = df1.fillna(df1.median())

In [26]:
df1 = pd.merge(df1 , df_position, on = 'id' , how = 'inner')

In [27]:
columns = df1.columns
remove = ('pos','teams','id','total_points')
a = set(columns)
b = a.difference(remove)
columns = list(b)

In [28]:
df1.loc[:,columns] = StandardScaler().fit_transform(df1.loc[:, columns])

In [29]:
X=df1.drop(['id'  ,'pos','teams'],axis ='columns') 


In [30]:
import pickle

with open('model_pickle','rb') as file:
    mp = pickle.load(file)

In [31]:
df1['predict'] = mp.predict(X)




In [32]:
df1 = pd.merge(df1 , df_id, on = 'id' , how = 'inner')

In [34]:
df_op = df1.loc[:,['predict' , 'name' , 'position']]

In [37]:
df_op.isnull().values.any()

False

In [39]:
df_op.to_csv('final_result')